In [56]:
from mip import *
import numpy as np
import time
from simulated_bifurcation import Ising
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

N_range = list(range(50,1001,50))
alpha_range = [1]

def generate_J_h(N_range,alpha_range):
    for N in N_range:
        for alpha in alpha_range:
            J = np.random.randint(-100,100,size=(N*alpha, N*alpha))/(10**6)
            J = (J + J.T)/2
            J = J - np.diag(np.diag(J))
            h = np.random.randint(-100,100,size=N*alpha)/(10**4)

            with open('test_gurobi/test_GB_J_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, J)

            with open('test_gurobi/test_GB_h_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, h)

            with open('test_gurobi/test_SB_J_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, J/4)

            with open('test_gurobi/test_SB_h_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, h/2 - sum([J[:,i]/8 for i in range(N*alpha)]) - sum([J[i,:]/8 for i in range(N*alpha)]))

generate_J_h(N_range,alpha_range)

In [57]:
def create_optimize_gurobi(J,h,J_SB,h_SB,max_seconds=-1):
    time_1 = time.time()
    size = J.shape[0]

    model = Model("Ising", solver_name=GRB)
    model.verbose = 0
    u = [model.add_var(var_type=BINARY) for _ in range(size)]
    z = [[model.add_var(var_type=BINARY) for _ in range(size)] for _ in range(size)]

    for i in range(size):
        for j in range(size):
            model += z[i][j] <= u[i]
            model += z[i][j] <= u[j]
            model += z[i][j] >= u[i] + u[j] -1

    model.objective = minimize((-1/2)*quicksum(quicksum(J[i,j]*z[i][j] for i in range(size)) for j in range(size)) + quicksum(h[i]*u[i] for i in range(size)))
    if max_seconds == -1:
        model.optimize(max_seconds=300)
    else:
        model.optimize(max_seconds=max_seconds)

    spins = [1 if u[i].x >= 0.99 else -1 for i in range(size)]
    energy = (-1/2)*sum([sum([J_SB[i,j]*spins[j] for j in range(size)])*spins[i] for i in range(size)]) + sum([h_SB[i]*spins[i] for i in range(size)])[0]

    processing_time = time.time() - time_1
    return (processing_time,np.array(spins),energy)

In [58]:
def create_optimize_SB(J,h):
    time_1 = time.time()

    model = Ising(J,h)
    model.optimize(convergence_threshold = 35,
        sampling_period = 60,
        time_step = 0.01, 
        symplectic_parameter = 2,
        agents = 5,
        ballistic=False,
        heated=True,
        final_pressure=1.,
        pressure_slope=0.2)

    processing_time = time.time() - time_1
    return(processing_time, model.ground_state, model.energy)

In [59]:
def test_protocol(N_range, alpha_range):
    energy_precision = {}
    proc_times = {}
    energy_precision_stop = {}

    for N in tqdm(N_range):
        energy_precision[N] = {}
        proc_times[N] = {}
        energy_precision_stop[N] = {}
        for alpha in alpha_range:
            with open('test_gurobi/test_SB_J_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_SB_J = np.load(f)

            with open('test_gurobi/test_SB_h_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_SB_h = np.expand_dims(np.load(f),axis=1)

            with open('test_gurobi/test_GB_J_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_GB_J = np.load(f)

            with open('test_gurobi/test_GB_h_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_GB_h = np.load(f)

            processing_time_sb, _, energy_sb = create_optimize_SB(test_SB_J,test_SB_h)
            processing_time_gb, _, energy_gb = create_optimize_gurobi(test_GB_J,test_GB_h,test_SB_J,test_SB_h)
            _, _, energy_gb_stopped = create_optimize_gurobi(test_GB_J,test_GB_h,test_SB_J,test_SB_h,processing_time_sb)

            energy_precision[N][alpha] = round((energy_sb - energy_gb)/abs(energy_gb),6)
            proc_times[N][alpha] = round(processing_time_gb/processing_time_sb,2)
            energy_precision_stop[N][alpha] = round((energy_gb_stopped - energy_sb)/abs(energy_sb),6)

    return energy_precision,proc_times,energy_precision_stop

In [79]:
energy_precision_1,proc_times_1,energy_precision_stop_1 = test_protocol(list(range(50,901,50)),[1])

  0%|          | 0/18 [00:00<?, ?it/s]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Bifurcated spins 0/5 Spins bifurcated in 0.183 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


  6%|▌         | 1/18 [00:00<00:13,  1.25it/s]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Bifurcated spins 0/5⠹ Bifurcated spins 5/5 Spins bifurcated in 0.253 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 11%|█         | 2/18 [00:02<00:23,  1.47s/it]

...

 89%|████████▉ | 16/18 [33:57<10:08, 304.10s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 2/5⠦ Bifurcated spins 4/5 Spins bifurcated in 2.237 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 94%|█████████▍| 17/18 [40:05<05:23, 323.57s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 2/5⠋ Bifurcated spins 3/5⠙ Bifurcated spins 3/5⠹ Bifurcated spins 4/5⠸ Bifurcated spins 4/5⠼ Bifurcated spins 4/5⠴ Bifurcated spins 4/5⠦ Bifurcated spins 4/5⠧ Bifurcated spins 4/5 Spins bifurcated in 3.15 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username


100%|██████████| 18/18 [46:24<00:00, 154.69s/it]


In [80]:
energy_precision_2,proc_times_2,energy_precision_stop_2 = test_protocol(list(range(50,401,50)),[2])

  0%|          | 0/8 [00:00<?, ?it/s]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Bifurcated spins 0/5 Spins bifurcated in 0.212 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 12%|█▎        | 1/8 [00:02<00:14,  2.09s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 2/5 Spins bifurcated in 0.382 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 25%|██▌       | 2/8 [00:09<00:31,  5.21s/it]

...

 75%|███████▌  | 6/8 [04:00<02:13, 66.99s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 2/5⠙ Bifurcated spins 4/5⠹ Bifurcated spins 4/5⠸ Bifurcated spins 4/5⠼ Bifurcated spins 4/5⠴ Bifurcated spins 4/5⠦ Bifurcated spins 4/5⠧ Bifurcated spins 4/5⠇ Bifurcated spins 4/5⠏ Bifurcated spins 4/5⠋ Bifurcated spins 4/5 Spins bifurcated in 2.563 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-

 88%|████████▊ | 7/8 [09:48<02:39, 159.13s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 1/5⠴ Bifurcated spins 2/5⠦ Bifurcated spins 3/5 Spins bifurcated in 2.24 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


100%|██████████| 8/8 [15:50<00:00, 118.77s/it]


In [81]:
energy_precision_3,proc_times_3,energy_precision_stop_3 = test_protocol(list(range(50,251,50)),[3])

  0%|          | 0/5 [00:00<?, ?it/s]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 1/5 Spins bifurcated in 0.283 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 20%|██        | 1/5 [00:03<00:13,  3.49s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 4/5 Spins bifurcated in 1.023 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 40%|████      | 2/5 [00:18<00:30, 10.26s/it]

...

 60%|██████    | 3/5 [00:53<00:42, 21.37s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 3/5⠇ Bifurcated spins 4/5 Spins bifurcated in 1.48 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 80%|████████  | 4/5 [03:05<01:05, 65.05s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 1/5⠙ Bifurcated spins 1/5⠹ Bifurcated spins 3/5⠸ Bifurcated spins 3/5 Spins bifurcated in 1.989 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


100%|██████████| 5/5 [08:58<00:00, 107.77s/it]


In [104]:
with open('test_gurobi/precisions_3.json') as json_file:
    precisions_dict = json.load(json_file)

with open('test_gurobi/precisions_early_stopping_3.json') as json_file:
    precisions_early_stopping_dict = json.load(json_file)

with open('test_gurobi/times_3.json') as json_file:
    times_dict = json.load(json_file)

In [105]:
times_dict

{'50': {'3': 2.67},
 '100': {'3': 6.68},
 '150': {'3': 11.93},
 '200': {'3': 65.08},
 '250': {'3': 148.71}}

In [106]:
precisions_early_stopping_dict

{'50': {'3': -0.0},
 '100': {'3': 0.940155},
 '150': {'3': 0.986043},
 '200': {'3': 0.968175},
 '250': {'3': 0.964593}}

In [107]:
precisions_dict

{'50': {'3': 0.0},
 '100': {'3': 0.0},
 '150': {'3': 7e-06},
 '200': {'3': 2.4e-05},
 '250': {'3': -0.075748}}